-----
### Klasyfikacja przy dowolnych podanych warunkach
<img src="../img/classify.png">

In [25]:
class From(object):
    def __init__(self, bayesian_network, test_data):
        self.bayesian_network = bayesian_network
        self.conditionals = []
        self.test_data = test_data
    
    def given(self, name, value):
        self.conditionals.append({"name": name, "value": value})
        return self
    
    def classify(self, class_attribute_name):
        final_result = []
        for class_state in self.find_class_attribute_states(class_attribute_name):
            result = 1
            temp_conditionals = copy.deepcopy(self.conditionals)
            temp_conditionals.append({"name": class_attribute_name, "value": class_state})
            
            for node in self.bayesian_network:
                tuples = []
                tuples.append((node["name"], self.get_conditional_value(node['name'], temp_conditionals), True))
                    
                for parent in node['parents']:    
                    tuples.append((parent['name'], self.get_conditional_value(parent['name'], temp_conditionals), False))
                
                multiplier = self.compute_probability(tuples)
                if multiplier != 0:
                    result *= multiplier
            
            final_result.append({"state": class_state, "value": result})  
                
        return self.normalize_probabilities_sum(final_result)
    
    def normalize_probabilities_sum(self, result_table):
        probabilities_sum = self.compute_probabilities_sum(result_table)
        
        if probabilities_sum == 0:
            return result_table
        
        for result in result_table:
            result['value'] = (result['value'] / probabilities_sum) * 100.0
        return result_table
    
    def compute_probabilities_sum(self, result_table):
        probabilities_sum = 0
        for result in result_table:
            probabilities_sum += result['value']
        return probabilities_sum
            
    def get_conditional_value(self, name, conditionals):
        for conditional in conditionals:
            if conditional["name"] == name:
                return conditional["value"]
        raise ValueError("There's no " + name + " conditional in conditionals")
    
    def find_class_attribute_states(self, class_attribute_name):
        for node in self.bayesian_network:
            if (node['name'] == class_attribute_name):
                return node['r']
            
        raise ValueError(class_attribute_name + " -> No class attribute state error")
    
    def compute_probability(self, tuples):
        all_conditions_counter = self.count_occurences_of_fullfilled_conditions(tuples)

        self.delete_parent_from_tuples(tuples)
        
        if len(tuples) == 0: 
            return all_conditions_counter / len(self.test_data)
        
        parent_conditions_counter = self.count_occurences_of_fullfilled_conditions(tuples)
     
        if all_conditions_counter == 0 or parent_conditions_counter == 0:
            return 1 / self.count_number_of_states(tuples)

        return all_conditions_counter / parent_conditions_counter
    
    def count_occurences_of_fullfilled_conditions(self, tuples):
        counter = 0
        for test_data_line in self.test_data:
            if (self.does_test_line_fullfill_conditions(tuples, test_data_line)):
                counter += 1
        return counter
    
    def does_test_line_fullfill_conditions(self, tuples, test_data_line):
        for condition_tuple in tuples:
            if test_data_line[condition_tuple[0]] != condition_tuple[1]:
                return False
        return True
    
    def count_number_of_states(self, tuples):
        result = 0
        for single_tuple in tuples:
            for node in self.bayesian_network:
                if node['name'] == single_tuple[0]:
                    result += len(node['r'])
        return result
    
    def delete_parent_from_tuples(self, tuples):
        for i, single_tuple in enumerate(tuples):
            if single_tuple[2]:
                del tuples[i]
        return tuples